In [1]:
import pandas as pd
import numpy as np
import sys

### Import Oracle Creator

In [2]:
sys.path.append("../../../../Tracer/src")
from DataHandling.TraceCreator import create_trace_matrix_from_trace_list

def create_trace_list(path_to_oracle):
    oracle = open(path_to_oracle).read().split("\n")
    trace_list = []
    for trace_item in oracle:
        if len(trace_item) > 0:
            source, targets = trace_item.split(":")
            for target in targets.split(" "):
                if len(target) != 0:
                    trace_list.append(" ".join([source, target]))
                else:
                    trace_list.append(" ".join([source, ""]))
    return create_trace_matrix_from_trace_list(trace_list)

# Original Oracles

In [3]:
r2d = pd.read_csv("../../Oracles/DataFrames/Level_1_to_Level_2.csv").set_index("id")

### Import New Oracle

In [4]:
d2d = create_trace_list("../Originals/TracedMatrices/ID_ID.txt")
d2d.head(3)

,47.txt,50.txt,40.txt,49.txt,33.txt,32.txt,34.txt,35.txt,48.txt,42.txt,46.txt,37.txt,45.txt,31.txt,39.txt,41.txt,38.txt,36.txt,44.txt,43.txt
id,,,,,,,,,,,,,,,,,,,,
47.txt,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50.txt,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40.txt,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### R 2 D

In [5]:
def get_similar_classes(c_id):
    similar_classes = []
    if c_id in d2d.index:
        for c2_id in d2d.columns:
            if d2d.loc[c_id][c2_id] == 1:
                similar_classes.append(c2_id)
    return similar_classes

In [6]:
missing_traces = []
n_missing_traces = 0
n_confirmed_traces = 0
n_undefined_classes = 0
table = r2d
for d_id in table.index:
    for c_id in table.columns:
        if table.loc[d_id][c_id] == 1:
            similar_classes = get_similar_classes(c_id)
            for s_class in similar_classes:
                if s_class not in table.columns:
                    n_undefined_classes = n_undefined_classes + 1
                elif s_class in table.columns and table.loc[d_id][s_class] == 0:
                    n_missing_traces = n_missing_traces + 1
                    missing_traces.append((d_id, s_class))
                elif s_class in table.columns and table.loc[d_id][s_class] == 1:
                    n_confirmed_traces = n_confirmed_traces + 1
                    
print("Number of Missing Traces: ", len(missing_traces))
print("NUmber of confirmed traces: ", n_confirmed_traces)
print("Number of undefined classes: ", n_undefined_classes)

Number of Missing Traces:  88
NUmber of confirmed traces:  490
Number of undefined classes:  0


# Add missing traces

In [7]:
for mt in missing_traces:
    r2d.loc[mt[0]][mt[1]] = 1

In [8]:
r2d.to_csv("../../Oracles/DataFrames/Level_1_to_Level_2.csv")
print("Done!")

Done!
